In [1]:
# Cell 1: Install required libraries
# Qwen3-VL requires latest transformers from GitHub
!pip uninstall transformers -y
!pip install git+https://github.com/huggingface/transformers --upgrade -q
!pip install -q accelerate bitsandbytes pillow pandas tqdm
print("✅ Libraries installed! Please restart the kernel if import fails.")

Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.1/516.1 kB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.8 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.8 MB/s eta 0:

In [2]:
# Cell 2: Import libraries and setup
import os
import re
import json
import torch
import pandas as pd
from PIL import Image
from tqdm import tqdm
from datetime import datetime
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
import transformers

print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Transformers version: 5.0.0.dev0
PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.83 GB


In [3]:
# Cell 3: Configuration
class Config:
    # Paths - UPDATE THESE FOR YOUR ENVIRONMENT
    IMAGE_DIR = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image"
    OUTPUT_DIR = "/kaggle/working"
    
    # Model
    MODEL_NAME = "Qwen/Qwen3-VL-8B-Instruct"
    
    # Processing
    CHECKPOINT_EVERY = 10  # Save checkpoint every N images
    MAX_NEW_TOKENS = 1024  # Response length
    
    # Output files
    OUTPUT_CSV = os.path.join(OUTPUT_DIR, "meme_analysis_cot.csv")
    CHECKPOINT_CSV = os.path.join(OUTPUT_DIR, "meme_analysis_checkpoint.csv")
    LOG_FILE = os.path.join(OUTPUT_DIR, "analysis_log.txt")

config = Config()
print(f"Image directory: {config.IMAGE_DIR}")
print(f"Output CSV: {config.OUTPUT_CSV}")

Image directory: /kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image
Output CSV: /kaggle/working/meme_analysis_cot.csv


In [4]:
# Cell 4: Load Model with 4-bit quantization
print("Loading Qwen3-VL-8B-Instruct with 4-bit quantization...")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForVision2Seq.from_pretrained(
    config.MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

processor = AutoProcessor.from_pretrained(
    config.MODEL_NAME,
    trust_remote_code=True
)

print(f"✅ Model loaded successfully!")
print(f"Model type: {type(model).__name__}")

Loading Qwen3-VL-8B-Instruct with 4-bit quantization...


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2258: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

✅ Model loaded successfully!
Model type: Qwen3VLForConditionalGeneration


In [5]:
# Cell 5: Define Chain-of-Thought Prompt Template

COT_PROMPT = """You are an expert multimodal AI assistant specialized in analyzing Bangladeshi political and social memes. You are shown a meme image.

## Think step by step through the following process before providing your final analysis:

1. **Scene Understanding:**
   - Carefully observe the setting and elements in the image.
   - Identify key objects, actions, and the general context.

2. **Character Identification:**
   - Determine if any recognizable figures (politicians, celebrities, cartoon characters) are present.
   - If unknown, describe the appearance and expression of the characters.

3. **Expression and Style Analysis:**
   - Examine the characters' facial expressions and body language.
   - Note any exaggerated or stylized features typical of memes.

4. **Text Extraction and Interpretation:**
   - Read and understand any text or captions included in the meme (may be in Bengali/Bangla).
   - Reflect on how the text interacts with the visuals (e.g., sarcasm, irony, humor).

5. **Message and Intent Analysis:**
   - Infer the main theme or message being conveyed.
   - Consider whether the meme is humorous, sarcastic, motivational, offensive, or neutral.

6. **Emotional and Social Tone:**
   - Assess the emotional tone (funny, mocking, inspirational, etc.).
   - Determine if it carries positive, negative, or mixed social implications.

7. **Visual Technique Check:**
   - Identify use of visual metaphors or exaggeration.
   - Comment on how these enhance or distort the message.

## After completing these reasoning steps internally, provide your final analysis in the following EXACT JSON format:

```json
{
    "Meme_Explanation": "A concise 1-2 sentence description of what the meme conveys and its humor/message",
    "Metaphor_Object": "The specific object, person, symbol, or text phrase that serves as the key metaphor (e.g., 'Sheikh Hasina', 'BNP logo', 'crying cat', 'bKash app', 'DUCSU election')",
    "Humor": "Choose ONE: Ironic | Mockery | Satirical | Other",
    "Metaphor": "Choose ONE: Both | Text | Image",
    "Political_Intensity": "Choose ONE: High | Moderate | Low"
}
```

IMPORTANT GUIDELINES:
- Output ONLY the JSON block, no additional text before or after.
- Use the EXACT labels provided in the choices.
- The meme may contain Bengali/Bangla text - translate and interpret it.
- Meme_Explanation: Keep it CONCISE. Focus on the core message/joke.
- Metaphor_Object: Must be a SPECIFIC object, person, symbol, logo, or phrase visible in the image or text. NOT abstract concepts. Examples: "Tiger Balm", "Shakib Al Hasan", "Awami League boat symbol", "Tom and Jerry", "15th August"
"""

print("✅ CoT Prompt template defined")
print(f"Prompt length: {len(COT_PROMPT)} characters")

✅ CoT Prompt template defined
Prompt length: 2625 characters


In [6]:
# Cell 6: Helper Functions

def log_message(message, log_file=None):
    """Print and optionally log a message."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_line = f"[{timestamp}] {message}"
    print(log_line)
    if log_file:
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write(log_line + "\n")

def extract_json_from_response(response_text):
    """Extract JSON from model response, handling various formats."""
    # Try to find JSON block in markdown code fence
    json_match = re.search(r'```(?:json)?\s*\n?({[\s\S]*?})\s*\n?```', response_text)
    if json_match:
        try:
            return json.loads(json_match.group(1))
        except json.JSONDecodeError:
            pass
    
    # Try to find raw JSON object
    json_match = re.search(r'({[\s\S]*"Humor"[\s\S]*})', response_text)
    if json_match:
        try:
            return json.loads(json_match.group(1))
        except json.JSONDecodeError:
            pass
    
    # If no JSON found, try parsing the whole response
    try:
        return json.loads(response_text.strip())
    except json.JSONDecodeError:
        return None

def parse_model_response(response_text, image_name):
    """Parse the model response and extract structured data."""
    result = {
        "Image_name": image_name,
        "Humor": "Other",
        "Metaphor": "Both",
        "Meme_Explanation": "",
        "Metaphor_Object": "",
        "Political_Intensity": "Low",
        "raw_response": response_text
    }
    
    parsed_json = extract_json_from_response(response_text)
    
    if parsed_json:
        # Map JSON fields to result
        field_mapping = {
            "Humor": ["Ironic", "Mockery", "Satirical", "Other"],
            "Metaphor": ["Both", "Text", "Image"],
            "Political_Intensity": ["High", "Moderate", "Low"]
        }
        
        for field, valid_values in field_mapping.items():
            if field in parsed_json:
                value = parsed_json[field].strip()
                # Find closest match
                for valid in valid_values:
                    if valid.lower() in value.lower() or value.lower() in valid.lower():
                        result[field] = valid
                        break
        
        # Copy text fields directly
        if "Meme_Explanation" in parsed_json:
            result["Meme_Explanation"] = parsed_json["Meme_Explanation"]
        if "Metaphor_Object" in parsed_json:
            result["Metaphor_Object"] = parsed_json["Metaphor_Object"]
    
    return result

def get_image_files(image_dir):
    """Get sorted list of image files."""
    valid_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp'}
    files = []
    for f in os.listdir(image_dir):
        if os.path.splitext(f)[1].lower() in valid_extensions:
            files.append(f)
    return sorted(files)

print("✅ Helper functions defined")

✅ Helper functions defined


In [7]:
# Cell 7: Main Analysis Function

def analyze_meme(image_path, model, processor):
    """Analyze a single meme image using CoT prompting."""
    
    # Load image
    image = Image.open(image_path).convert('RGB')
    
    # Resize if too large
    max_size = 512
    if max(image.size) > max_size:
        ratio = max_size / max(image.size)
        new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
        image = image.resize(new_size, Image.LANCZOS)
    
    # Prepare messages for Qwen3-VL
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": COT_PROMPT}
            ]
        }
    ]
    
    # Process inputs using apply_chat_template
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device)
    
    # Generate response
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=config.MAX_NEW_TOKENS,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    
    # Decode response
    generated_ids = [out[len(inp):] for inp, out in zip(inputs.input_ids, output_ids)]
    response = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

print("✅ Analysis function defined")

✅ Analysis function defined


In [8]:
# Cell 8: Test on a single image

# Get list of images
image_files = get_image_files(config.IMAGE_DIR)
print(f"Found {len(image_files)} images in {config.IMAGE_DIR}")

if len(image_files) > 0:
    # Test on first image
    test_image = image_files[0]
    test_path = os.path.join(config.IMAGE_DIR, test_image)
    
    print(f"\n{'='*60}")
    print(f"Testing on: {test_image}")
    print(f"{'='*60}")
    
    response = analyze_meme(test_path, model, processor)
    print(f"\n📝 Model Response:\n{response}")
    
    parsed = parse_model_response(response, test_image)
    print(f"\n✅ Parsed Result:")
    for key, value in parsed.items():
        if key != 'raw_response':
            print(f"  {key}: {value}")
else:
    print("❌ No images found!")

Found 330 images in /kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image

Testing on: test0001.jpg

📝 Model Response:
```json
{
    "Meme_Explanation": "A meme depicting a politician's exaggerated disappointment after losing a DUCSU election, humorously contrasting with the public's apathy.",
    "Metaphor_Object": "DUCSU election",
    "Humor": "Ironic",
    "Metaphor": "Text",
    "Political_Intensity": "Moderate"
}
```

✅ Parsed Result:
  Image_name: test0001.jpg
  Humor: Ironic
  Metaphor: Text
  Meme_Explanation: A meme depicting a politician's exaggerated disappointment after losing a DUCSU election, humorously contrasting with the public's apathy.
  Metaphor_Object: DUCSU election
  Political_Intensity: Moderate


In [9]:
# Cell 9: Process All Images with Checkpointing

def process_all_images(image_dir, model, processor, config, resume_from=0):
    """Process all images with checkpointing and logging."""
    
    image_files = get_image_files(image_dir)
    total_images = len(image_files)
    
    log_message(f"Starting analysis of {total_images} images", config.LOG_FILE)
    log_message(f"Checkpoint every {config.CHECKPOINT_EVERY} images", config.LOG_FILE)
    
    # Load existing checkpoint if resuming
    results = []
    if resume_from > 0 and os.path.exists(config.CHECKPOINT_CSV):
        existing_df = pd.read_csv(config.CHECKPOINT_CSV, encoding='utf-8-sig')
        results = existing_df.to_dict('records')
        log_message(f"Resuming from checkpoint with {len(results)} existing results", config.LOG_FILE)
    
    # Process images
    for idx in tqdm(range(resume_from, total_images), desc="Analyzing memes"):
        image_name = image_files[idx]
        image_path = os.path.join(image_dir, image_name)
        
        try:
            log_message(f"[{idx+1}/{total_images}] Processing: {image_name}", config.LOG_FILE)
            
            # Analyze image
            response = analyze_meme(image_path, model, processor)
            
            # Parse response
            result = parse_model_response(response, image_name)
            results.append(result)
            
            # Log parsed result
            log_message(f"  → Humor: {result['Humor']}, Political: {result['Political_Intensity']}", config.LOG_FILE)
            log_message(f"  → Metaphor: {result['Metaphor']}, Object: {result['Metaphor_Object'][:50] if result['Metaphor_Object'] else 'N/A'}...", config.LOG_FILE)
            
            # Save checkpoint
            if (idx + 1) % config.CHECKPOINT_EVERY == 0:
                checkpoint_df = pd.DataFrame(results)
                checkpoint_df.to_csv(config.CHECKPOINT_CSV, index=False, encoding='utf-8-sig')
                log_message(f"💾 Checkpoint saved at image {idx+1}", config.LOG_FILE)
            
            # Clear GPU cache periodically
            if (idx + 1) % 5 == 0:
                torch.cuda.empty_cache()
                
        except Exception as e:
            log_message(f"❌ Error processing {image_name}: {str(e)}", config.LOG_FILE)
            # Add placeholder result
            results.append({
                "Image_name": image_name,
                "Humor": "Other",
                "Metaphor": "Both",
                "Meme_Explanation": f"Error: {str(e)}",
                "Metaphor_Object": "Unknown",
                "Political_Intensity": "Low",
                "raw_response": ""
            })
    
    # Final save
    final_df = pd.DataFrame(results)
    final_df.to_csv(config.OUTPUT_CSV, index=False, encoding='utf-8-sig')
    log_message(f"✅ Analysis complete! Results saved to {config.OUTPUT_CSV}", config.LOG_FILE)
    
    return final_df

print("✅ Batch processing function defined")

✅ Batch processing function defined


In [10]:
# Cell 10: Run Full Analysis

# Set resume_from to continue from a specific image index (0 to start fresh)
RESUME_FROM = 0

print(f"{'='*60}")
print("STARTING FULL MEME ANALYSIS")
print(f"{'='*60}")

results_df = process_all_images(
    image_dir=config.IMAGE_DIR,
    model=model,
    processor=processor,
    config=config,
    resume_from=RESUME_FROM
)

print(f"\n{'='*60}")
print("ANALYSIS COMPLETE!")
print(f"{'='*60}")
print(f"Total images processed: {len(results_df)}")
print(f"Output saved to: {config.OUTPUT_CSV}")

STARTING FULL MEME ANALYSIS
[2025-12-01 06:56:35] Starting analysis of 330 images
[2025-12-01 06:56:35] Checkpoint every 10 images


Analyzing memes:   0%|          | 0/330 [00:00<?, ?it/s]

[2025-12-01 06:56:35] [1/330] Processing: test0001.jpg


Analyzing memes:   0%|          | 1/330 [00:11<1:05:30, 11.95s/it]

[2025-12-01 06:56:47]   → Humor: Ironic, Political: High
[2025-12-01 06:56:47]   → Metaphor: Text, Object: DUCSU election...
[2025-12-01 06:56:47] [2/330] Processing: test0002.jpg


...

[2025-12-01 08:05:37]   → Humor: Ironic, Political: Moderate
[2025-12-01 08:05:37]   → Metaphor: Text, Object: The man in the blue hoodie with sunglasses...
[2025-12-01 08:05:37] [329/330] Processing: test0329.jpg


Analyzing memes: 100%|█████████▉| 329/330 [1:09:14<00:12, 12.32s/it]

[2025-12-01 08:05:49]   → Humor: Mockery, Political: High
[2025-12-01 08:05:49]   → Metaphor: Text, Object: Awami League logo...
[2025-12-01 08:05:49] [330/330] Processing: test0330.jpg


Analyzing memes: 100%|██████████| 330/330 [1:09:26<00:00, 12.62s/it]

[2025-12-01 08:06:01]   → Humor: Mockery, Political: High
[2025-12-01 08:06:01]   → Metaphor: Text, Object: Elias Hossain...
[2025-12-01 08:06:01] 💾 Checkpoint saved at image 330
[2025-12-01 08:06:01] ✅ Analysis complete! Results saved to /kaggle/working/meme_analysis_cot.csv

ANALYSIS COMPLETE!
Total images processed: 330
Output saved to: /kaggle/working/meme_analysis_cot.csv


In [11]:
# Cell 11: View Results Summary

# Load results
if os.path.exists(config.OUTPUT_CSV):
    df = pd.read_csv(config.OUTPUT_CSV, encoding='utf-8-sig')
    
    print(f"Total entries: {len(df)}")
    print(f"\nColumns: {list(df.columns)}")
    
    print(f"\n{'='*40}")
    print("DISTRIBUTION SUMMARY")
    print(f"{'='*40}")
    
    print(f"\n📊 Humor Distribution:")
    print(df['Humor'].value_counts())
    
    print(f"\n📊 Political Intensity Distribution:")
    print(df['Political_Intensity'].value_counts())
    
    print(f"\n📊 Metaphor Distribution:")
    print(df['Metaphor'].value_counts())
    
    print(f"\n\n📋 First 5 entries (preview):")
    display(df[['Image_name', 'Humor', 'Political_Intensity', 'Metaphor', 'Metaphor_Object']].head())
else:
    print("No results file found. Run the analysis first.")

Total entries: 330

Columns: ['Image_name', 'Humor', 'Metaphor', 'Meme_Explanation', 'Metaphor_Object', 'Political_Intensity', 'raw_response']

DISTRIBUTION SUMMARY

📊 Humor Distribution:
Humor
Ironic       204
Mockery       80
Satirical     46
Name: count, dtype: int64

📊 Political Intensity Distribution:
Political_Intensity
High        191
Low         118
Moderate     21
Name: count, dtype: int64

📊 Metaphor Distribution:
Metaphor
Image    165
Text     132
Both      33
Name: count, dtype: int64


📋 First 5 entries (preview):


,Image_name,Humor,Political_Intensity,Metaphor,Metaphor_Object
0,test0001.jpg,Ironic,High,Text,DUCSU election
1,test0002.jpg,Ironic,Low,Image,Football player
2,test0003.jpg,Ironic,High,Image,Tom (cartoon character)
3,test0004.jpg,Ironic,High,Image,Hulk
4,test0005.jpg,Ironic,Moderate,Image,The man's skeptical expression


In [12]:
# Cell 12: Create Metadata CSV (Main columns only)

if os.path.exists(config.OUTPUT_CSV):
    df = pd.read_csv(config.OUTPUT_CSV, encoding='utf-8-sig')
    
    # Select only the required columns for metadata
    metadata_columns = ['Image_name', 'Humor', 'Metaphor', 'Meme_Explanation', 'Metaphor_Object', 'Political_Intensity']
    metadata_df = df[metadata_columns]
    
    # Save metadata file
    metadata_path = os.path.join(config.OUTPUT_DIR, "metadata.csv")
    metadata_df.to_csv(metadata_path, index=False, encoding='utf-8-sig')
    
    print(f"✅ Metadata file created: {metadata_path}")
    print(f"Rows: {len(metadata_df)}")
    print(f"Columns: {list(metadata_df.columns)}")
    
    print(f"\n📋 Preview:")
    display(metadata_df.head(10))
else:
    print("No results file found. Run the analysis first.")

✅ Metadata file created: /kaggle/working/metadata.csv
Rows: 330
Columns: ['Image_name', 'Humor', 'Metaphor', 'Meme_Explanation', 'Metaphor_Object', 'Political_Intensity']

📋 Preview:


,Image_name,Humor,Metaphor,Meme_Explanation,Metaphor_Object,Political_Intensity
0,test0001.jpg,Ironic,Text,A meme depicting a politician's exaggerated di...,DUCSU election,High
1,test0002.jpg,Ironic,Image,A meme showing a dejected football player with...,Football player,Low
2,test0003.jpg,Ironic,Image,A meme depicting Tom from Tom and Jerry being ...,Tom (cartoon character),High
3,test0004.jpg,Ironic,Image,A meme showing the Hulk crying while being hel...,Hulk,High
4,test0005.jpg,Ironic,Image,A meme mocking the perception of Bangladesh's ...,The man's skeptical expression,Moderate
5,test0006.jpg,Satirical,Text,A meme comparing four types of people in Bangl...,Four individuals labeled with Bengali terms fo...,Low
6,test0007.jpg,Ironic,Text,A sarcastic jab at politicians who claim to al...,Deuliya Rashtro,High
7,test0008.jpg,Ironic,Text,A meme mocking political figures by juxtaposin...,A Man Without a Vote is a Man without Protection,High
8,test0009.jpg,Ironic,Image,A seal expresses outrage at 'posting digits' a...,Seal,Low
9,test0010.jpg,Ironic,Image,The meme humorously depicts a man's exasperate...,The man in the car with a confused expression,Low
